In [1]:
import pandas as pd
import numpy as np
import shap
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap

# 设置全局字体
rcParams['font.family'] = 'Times New Roman'
rcParams['font.size'] = 24

# 自定义红-紫-蓝渐变色
custom_cmap = LinearSegmentedColormap.from_list("custom_bwr", ['#3D72E6', '#9A51E6', '#EA3B8F'])

# ------------------- 配置 -------------------
# 特征选择和重命名
features = ['sex', 'age', 'edu', 'city', 'hunyin', 'fangchan', 'manbing', 'qushi',
            'shecan', 'ziqian', 'jianmian','lngdp','fuyangbi','time']
features_renamed = ['x' + str(i+1) for i in range(len(features))]
features_labels = [
    'Sex',                    # sex
    'Age',                    # age
    'Education Level',        # edu
    'Residence',  # city
    'Marital Status',         # hunyin
    'Property Assets',      # fangchan
    'Chronic Disease',        # manbing
    'Bereavement',         # qushi
    'Social Participation',   # shecan
    'Financial Support from Children',     # ziqian
    'Visit Frequency of Children',      # jianmian
    'GDP',                    # lngdp
    'Old-Age Dependency Ratio',                # fuyangbi
    'Survey Time'       # time
]
feature_mapping = dict(zip(features, features_renamed))
# ------------------- 图形函数 -------------------
def plot_shap_beeswarm_like(shap_values, X_used, features_labels, filename='beeswarm_plot.pdf'):
    shap_df = pd.DataFrame(shap_values, columns=X_used.columns)[X_used.columns]
    shap_df.columns = features_labels
    X_used_df = X_used.copy()
    X_used_df.columns = features_labels

    shap_long = shap_df.melt(var_name='Feature', value_name='SHAP Value')
    X_long = X_used_df.melt(var_name='Feature', value_name='Feature Value')

    shap_long['Feature'] = pd.Categorical(shap_long['Feature'], categories=features_labels[::-1], ordered=True)
    X_long['Feature'] = pd.Categorical(X_long['Feature'], categories=features_labels[::-1], ordered=True)

    merged = pd.concat([shap_long, X_long['Feature Value']], axis=1)

    fig, ax = plt.subplots(figsize=(14, 10))

    for idx in range(len(features_labels)):
        ax.axhline(y=idx, color='#E0E0E0', linestyle='-', linewidth=1, zorder=0)

    for idx, feature in enumerate(features_labels[::-1]):
        df_f = merged[merged["Feature"] == feature]
        x_vals = df_f["SHAP Value"].values
        f_vals = df_f["Feature Value"].values
        y_vals = np.random.normal(loc=idx, scale=0.15, size=len(x_vals))
        norm = plt.Normalize(np.nanmin(f_vals), np.nanmax(f_vals))
        rgba_vals = custom_cmap(norm(f_vals))
        rgba_vals[pd.isnull(f_vals)] = np.array([176/255, 176/255, 176/255, 1.0])
        ax.scatter(x_vals, y_vals, c=rgba_vals, s=25, alpha=0.7, zorder=1)

    ax.set_yticks(range(len(features_labels)))
    ax.set_yticklabels(features_labels[::-1])
    ax.tick_params(axis='y', left=False)
    ax.spines[['top', 'right', 'left']].set_visible(False)
    ax.axvline(0, color='gray', linestyle='--')
    ax.set_xlabel('SHAP value (impact on model output)')
    ax.set_ylabel('')
    ax.set_xlim(-2, 2)

    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('Feature value')
    plt.tight_layout()
    plt.savefig(filename, dpi=600, bbox_inches='tight')
    plt.close()

def plot_shap_bubble(shap_values, features_labels, filename='bubble_plot.pdf'):
    shap_array = np.abs(np.array(shap_values))
    mean_importance = shap_array.mean(axis=0)

    importance_dict = dict(zip(features_labels, mean_importance))
    ordered_features = features_labels[::-1]
    ordered_importance = [importance_dict[f] for f in ordered_features]

    fig, ax = plt.subplots(figsize=(3, 10))
    y_pos = np.arange(len(ordered_features))
    x_vals = np.zeros_like(y_pos)
    bubble_sizes = np.array(ordered_importance) * 1500
    bubble_colors = ['#9584C1'] * len(ordered_features)

    ax.scatter(x_vals, y_pos, s=bubble_sizes, color=bubble_colors,
               edgecolors='black', linewidths=1.0, alpha=0.9)
    ax.set_ylim(-1, len(ordered_features))
    ax.set_xlim(-1, 1)
    ax.set_xticks([])
    ax.set_yticks(y_pos)
    ax.set_yticklabels(ordered_features)
    ax.set_ylabel('Feature (same order as beeswarm)')
    plt.tight_layout()
    plt.savefig(filename, dpi=600, bbox_inches='tight')
    plt.close()

# ------------------- 处理某个区域的函数 -------------------
def process_region_group(region_value, data):
    print(f"Processing region {region_value}...")

    sub_data = data[data['region'] == region_value].copy()
    sub_data = sub_data.dropna(subset=list(feature_mapping.values()) + ['qishi'])

    X = sub_data[list(feature_mapping.values())]
    y = sub_data['qishi']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    lin_regr = linear_model.LinearRegression()
    lin_regr.fit(X_train, y_train)

    X_train_summary = shap.kmeans(X_train, 10)
    explainer = shap.KernelExplainer(lin_regr.predict, X_train_summary)
    shap_values = explainer.shap_values(X_test)

    beeswarm_file = f'beeswarm_region{region_value}.pdf'
    bubble_file = f'bubble_region{region_value}.pdf'

    plot_shap_beeswarm_like(shap_values, X_test, features_labels, filename=beeswarm_file)
    plot_shap_bubble(shap_values, features_labels, filename=bubble_file)

    print(f"Region {region_value} done. Saved {beeswarm_file} and {bubble_file}")

# ------------------- 主程序入口 -------------------
if __name__ == '__main__':
    print("Reading data...")
    data = pd.read_stata('/kaggle/input/data-0522/data.dta')
    data = data.rename(columns=feature_mapping)

    for region_id in [1, 2, 3, 4]:
        process_region_group(region_id, data)

    print("All regions processed!")


Reading data...
Processing region 1...


  0%|          | 0/947 [00:00<?, ?it/s]

Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.


Region 1 done. Saved beeswarm_region1.pdf and bubble_region1.pdf
Processing region 2...


  0%|          | 0/592 [00:00<?, ?it/s]

...

Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.


Region 3 done. Saved beeswarm_region3.pdf and bubble_region3.pdf
Processing region 4...


  0%|          | 0/234 [00:00<?, ?it/s]

Region 4 done. Saved beeswarm_region4.pdf and bubble_region4.pdf
All regions processed!


Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
